In [1]:
from load_data import *
from partition import *
from normalise import *
from exacerbation_periods import *
from smooth import *
from plot_helpers import *
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go


#Settings
datadir = "../../../../SmartCareData/"
plotsdir="../../../../PlotsSmartcare/O2_FEV1/"

In [2]:
# Process O2-FEV1 data
O2_FEV1 = create_O2_FEV1_df(datadir)

# Compute FEV1 % Predicted
O2_FEV1['FEV1 % Predicted'] = O2_FEV1.apply(lambda x: x["FEV1"] / x["Predicted FEV1"] * 100, axis=1)

# Partition FEV1 % Predicted in 3 groups
O2_FEV1["FEV1 % Predicted Group"] = partition_in_n_equal_groups(O2_FEV1['FEV1 % Predicted'],3)

# What are we analysing?
O2_FEV1.head()

/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/O2-FEV1 analysis/load_data.py:11: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  measurements = pd.read_csv(datadir + "mydata.csv").rename(columns={"FEV 1": "FEV1"})
/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


Removed 11 measurements where O2 Sat > 100%, kept 123136 measurements
O2 Saturation contains 13418 measurements
FEV1 contains 10418 measurements
Removed 4684 rows with O2_FEV1 inner join, kept 68% of measurements (10100)
Removed 1008 duplicates, 9092 measurements left


,ID,Date recorded,O2 Saturation,FEV1,Age,Sex,Height,Weight,Predicted FEV1,FEV1 Set As,Pulmonary Exacerbation,Less Exacerbation,Transplant Recipients,Date Last PE Start,Date Last PE Stop,Comments,FEV1 % Predicted,FEV1 % Predicted Group
0,79,2015-09-10,95.0,1.53,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,40.691489,<42.7
1,79,2015-09-11,97.0,1.68,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,44.680851,[42.7;66.0[
2,79,2015-09-12,97.0,1.48,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,39.361702,<42.7
3,79,2015-08-13,95.0,1.63,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,43.351064,[42.7;66.0[
4,79,2015-11-07,94.0,1.52,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,40.425532,<42.7


# O2-FEV1 analysis
## Definitions
- O2 Saturation
- FEV 1
- Predicted FEV 1 in %: measure for the amount of airway obstruction either due to sputum load (non-permanent obstruction) or scars in the lungs (permanent obstruction).

## Literature
[The association between forced expiratory volume in one second (FEV1) and pulse oximetric measurements of arterial oxygen saturation (SpO2) in the patients with COPD: A preliminary study](https://pubmed.ncbi.nlm.nih.gov/24949035/).
- Context: 31 patients with COPD
- Key results: There was not statistically significant correlation between FEV1 % predicted and SpO2 values (P < 0.05), but a great correlation existed between FEV1/FVC % predicted and SpO2 values (r = 0.556, P < 0.001).

 [ANALYSIS OF CORRELATION BETWEEN FEV1/FEV6 AND OXYGEN SATURATION DURING SIX-MINUTE WALK TEST (6MWT) IN COPD PATIENTS](https://www.researchgate.net/publication/351322676_ANALYSIS_OF_CORRELATION_BETWEEN_FEV1FEV6_AND_OXYGEN_SATURATION_DURING_SIX-MINUTE_WALK_TEST_6MWT_IN_COPD_PATIENTS)
 - Need to download

Note: nobody segments the input by groups. Probably because they don't have enough data to do this.

In [ ]:
## Raw scatter plots

In [ ]:
# Raw plot without trendline
fig = px.scatter(O2_FEV1, y="O2 Saturation", x="FEV1")
# fig = px.scatter(O2_FEV1, y="O2 Saturation", x="FEV1", trendline="ols")
fig.update_layout(autosize=False, width=500, height=500)
fig.show()
fig.write_image(plotsdir + "FEV1-O2 raw.pdf")

fig = px.scatter(O2_FEV1, y="O2 Saturation", x="FEV1 % Predicted")
fig.update_layout(autosize=False, width=500, height=500)
fig.show()
fig.write_image(plotsdir + "FEV1 % Predicted-O2 raw.pdf")

O2_FEV1.shape

In [ ]:
var = 'FEV1'; xaxis_range = [0, 5.1]
# var = 'FEV1 % Predicted'; xaxis_range = [0, 155]

# Plot with height
O2_FEV1['Height Group'], height_labels=partition_in_n_equal_groups(O2_FEV1['Height'], 3, True)

fig = px.scatter(O2_FEV1, y="O2 Saturation", x=var, color="Height Group", marginal_x="histogram", marginal_y="histogram")
# fig.update_layout(autosize=False, width=800, height=800)
fig.update_traces(marker=dict(size=5),
                  selector=dict(mode='markers'))
# fig.update_xaxes(range=[0.9*min(O2_FEV1["FEV1"]), 1.1*max(O2_FEV1["FEV1"])])
# fig.update_yaxes(range=[0.9*min(O2_FEV1["O2 Saturation"]), max(1.1*O2_FEV1["O2 Saturation"])])
# fig.show()

for height_group in height_labels:
    mask_height=O2_FEV1['Height Group'] == height_group

    fig = px.scatter(O2_FEV1[mask_height], y="O2 Saturation", x=var,
                     title='Height group {}'.format(height_group))
    fig.update_layout(autosize=False, width=500, height=500)
    fig.update_traces(marker=dict(size=5),
                      selector=dict(mode='markers'))
    fig.update_xaxes(range=xaxis_range)
    fig.update_yaxes(range=[74, 102])
    fig.show()
    filename= "Height group " +  height_group + " " + var + "-O2 raw"
    fig.write_image(plotsdir + filename + ".pdf")

In [ ]:
predicted = False
x_var = 'Weight'
lung_function_var = 'FEV1 % Predicted' if predicted else 'FEV1'

fig1, fig2, _ = desaturation_FEV1_for_variable(O2_FEV1, x_var, n_var_groups=6, predicted=predicted)
fig1.show()
fig1.write_image(plotsdir + "Desaturation {} for {} main plot.pdf".format(lung_function_var, x_var))

fig2.show()
fig2.write_image(plotsdir + "Desaturation {} for {} with lines.pdf".format(lung_function_var, x_var))

# Observations
# o2 10, var 15 contains a lot of noise and groups have very low number of datapoints. It gets too patient specific

## Per patient plots

In [ ]:
# Same patient variability: can we observe the same pattern for each patient?
O2_FEV1.head()

# Include time evolution into the graph
def to_float(dt_time):
    return (365*dt_time.year + 12*dt_time.month + dt_time.day)/12

mindate = to_float(min(O2_FEV1["Date recorded"]))
O2_FEV1["Months since study start"]=O2_FEV1["Date recorded"].apply(lambda x: to_float(x)-mindate)



In [ ]:
# Same patient variability: can we observe the same pattern for each patient?
IDs=O2_FEV1.ID.unique()
for ID in IDs:
    plot_o2_fev1_raw_for_id(O2_FEV1, ID)

## O2-FEV1 analysis summary plot

In [ ]:
# Add exacerbated labels
# Compute exacerbation labels
# O2_FEV1_processed = compute_ex_labels_from_heuristics(antibioticsdata, patientsdata, O2_FEV1)

# Get exacerbation labels from the predictive classifier
O2_FEV1_processed = merge_pred_ex_labels_to(O2_FEV1, get_pred_ex_labels(datadir))

# Smooth
O2_FEV1_processed = smooth_max(O2_FEV1_processed, ['FEV1', 'FEV1 % Predicted'])
O2_FEV1_processed = smooth_avg(O2_FEV1_processed, ['O2 Saturation'])
print("Removed {} nan entries after max smoothing, kept {}".format(O2_FEV1_processed.isna().sum().max(), O2_FEV1_processed.shape[0]))
O2_FEV1_processed.dropna(inplace=True)

# Normalise
O2_FEV1_processed = norm_by_stable_baseline(O2_FEV1_processed,
                                            ['O2 Saturation', 'O2 Saturation smoothed', 'FEV1', 'FEV1 % Predicted', 'FEV1 smoothed', 'FEV1 % Predicted smoothed'])

# Settings
prefix, ex_column, x, y = set_x_y_vars(
      with_predicted_labels=True,
      with_predicted_fev1=True,
      is_smoothed_fev=True,
      is_smoothed_o2=False,
      is_normalised=False,
  )

is_partitioned=True
if not is_partitioned:
  # Plot O2-FEV with respective displots
  fig=plot_o2_fev_with_displots(O2_FEV1_processed, x, y, ex_column)
  fig.show()
  fig.write_image("{}/Ex labels {} {}-{}.pdf".format(plotsdir, prefix, x, y), width=1400, height=600)

  # Plot subsampled O2-FEV scatter
  fig = plot_subsampled_scatter(x, y, O2_FEV1_processed, random_state=7)
  fig.show()
else:

  fev1_prct_thresholds = [60, 70, 90]
  O2_FEV1_processed['FEV1 % Predicted Group'] = partition_given_thresholds(O2_FEV1['FEV1 % Predicted'], fev1_prct_thresholds)
  # O2_FEV1_processed["FEV1 % Predicted Group"] = partition_in_n_equal_groups(O2_FEV1['FEV1 % Predicted'], 3)

  # Create a for loop for each FEV1 % Predicted Group
  for fev_group in O2_FEV1_processed['FEV1 % Predicted Group'].unique():
    mask = O2_FEV1_processed['FEV1 % Predicted Group'] == fev_group

    # Count patients
    n_patients=len(O2_FEV1_processed[mask].ID.unique())

    # Plot
    fig=plot_o2_fev_with_displots(O2_FEV1_processed[mask], x, y, ex_column)
    fig.update_layout(title="{} patients, FEV1 % Predicted {}".format(n_patients, fev_group))
    fig.show()
    fig.write_image("{}/Ex labels {} {}-{} group {}.pdf".format(plotsdir, prefix, x, y, fev_group), width=1400, height=600)

    # Plot subsampled O2-FEV scatter
    # fig = plot_subsampled_scatter(x, y, O2_FEV1_processed[mask], random_state=7)
    # fig.show()

## Test the sensitivity of random states on subsampled plots
We wonder if different random states will give largely different results, to an extend that it could challenge the model assumptions.
It turns out that, despite an effect on the x axis range, the subsampled plots are very similar.

In [ ]:
# Add subsampled plot
prefix, ex_column, x, y = set_x_y_vars(with_predicted_labels=True, with_predicted_fev1=True, is_smoothed=True, is_normalised=False)

for random_state in range(1,10):
  print(random_state)
  fig = plot_subsampled_scatter(x, y, O2_FEV1_processed, random_state=random_state)
  fig.show()

## Full sized displots

In [ ]:
# This part fo the code is unused now
# But the displots can be useful for the report, so I keep it here

if predicted:
    if normalised:
        xrange=[-1.8, 1.2]
    else:
        xrange=[0, 155]
    bin_size=4
else:
    if normalised:
        xrange=[-42,40]
    else:
        xrange=[0, 5.1]
    bin_size=0.14

# Raw plot with superposed exacerbation labels
fig = px.scatter(O2_FEV1_processed, y=y, x=x, color=ex_column)
# fig.update_layout(autosize=False, width=800, height=800)
fig.update_traces(marker=dict(size=5),
                  selector=dict(mode='markers'))
# fig.show()
# fig.write_image("{}/{}-{} by exacerbated labels superposed raw.pdf".format(plotsdir, x, y))

# Raw individual plot for each exacerbation label]
fig_scatter = px.scatter(O2_FEV1_processed.dropna(), y=y, x=x,
                 # color='Exacerbation Labels',
                 facet_col=ex_column)
fig_scatter.update_layout(autosize=False, width=1000, height=500)
fig_scatter.update_traces(marker=dict(size=5),
                  selector=dict(mode='markers'))
# fig_scatter.update_xaxes(range=xrange)
if not(normalised): fig_scatter.update_yaxes(range=[74, 102])
# fig_scatter.show()
# fig.write_image("{}/Ex labels {} {}-{} individual raw.pdf".format(plotsdir, prefix, x, y), width=1300, height=400)

def distplot(x, group_labels, bin_size):
    data=[]
    for exacerbation_label in group_labels:
        mask = O2_FEV1_processed[ex_column] == exacerbation_label
        data.append(O2_FEV1_processed[x][ mask ])
        print("{} measurements with {} = {}".format(sum(mask), ex_column, exacerbation_label))
    fig = ff.create_distplot(data, group_labels, bin_size=bin_size, show_rug=False)
    fig.update_xaxes(title_text=x)
    fig.update_layout(autosize=False, width=1000, height=400)
    return fig

# Create distplot for x = lung function
group_labels = [True, False] if with_predicted_labels else ["Exacerbation Period", "Stable Period"]
fig_distplot_fev1 = distplot(x, group_labels, bin_size)
fig_distplot_fev1.show()
# fig.write_image("{}/Ex labels {} {} distplot.pdf".format(plotsdir, prefix, x), width=1000, height=400)

# Create distplot for y = O2 saturation
group_labels = [True, False] if with_predicted_labels else ["Exacerbation Period", "Stable Period"]
fig_displot_o2 = distplot(y, group_labels, 1)
fig_displot_o2.show()
# fig_displot_o2.write_image("{}/Ex labels {} {} distplot.pdf".format(plotsdir, prefix, y), width=1000, height=400)

## Patient level plots to validate or validate outliers down

In [ ]:
# O2-FEV1: Outliers down can happen, however if you are persistently getting low reading it means that it's not an outlier
# Checking O2 sat stable period measurements outliers
exacerbation_label = 'Stable Period'

# Case 1:
IDs = ['180', '202']
o2_saturation_threshold = 87

# Case 2: smoothed
o2_saturation_threshold = 90

# Plot
ids = O2_FEV1_processed[ (O2_FEV1_processed['Exacerbation Labels'] == exacerbation_label) & (O2_FEV1_processed['O2 Saturation'] < o2_saturation_threshold) ].ID.unique()
print(ids)
for id in ids:
    plot_o2_fev1_raw_for_id(O2_FEV1_processed, id, save=False, show=True)

## How to smooth O2?

In [ ]:
# Plots O2 saturation against Date for a given patient id
def plot_o2_saturation_for_id(df, id, plotsdir, save=False, show=False):
    mask = df['ID'] == id
    fig = px.scatter(df[mask], x='Date recorded', y='O2 Saturation')
    fig.update_layout(autosize=False, width=1000, height=400)
    fig.update_traces(marker=dict(size=5),
                      selector=dict(mode='markers'))
    if save:
        fig.write_image("{}/O2 for ID/O2 saturation for ID {}.pdf".format(plotsdir, id), width=1000, height=400)
    if show:
        fig.show()
        
# For each patient id, plot O2 saturation against Date recorded
for id in O2_FEV1.ID.unique():
    plot_o2_saturation_for_id(O2_FEV1, id, plotsdir, save=True, show=False)

# Create Factor Functions
We want the characterise the factor function that links the Unblocked FEV1 with its parents: Healthy FEV1 and Lung Damage. Here's a model of the relations betweeen those three varaibles: the unblocked FEV1 (L) of an individual is the healthy FEV1 (L), the theoretical lung function based on height, DOB, gdner, ethnicity (TBC exactly which), pejorated by the % in lung damage

In [58]:
# Factor function for unblocked FEV1

df_unblocked_factor=pd.DataFrame(columns=['ID,', 'Unblocked FEV1', 'Healthy FEV1'])
for id in O2_FEV1.ID.unique():
  # For a given patient id, filter the FEV1 measurements
  mask = O2_FEV1['ID'] == id
  O2_FEV1_patient = O2_FEV1[mask]
  # Find the unblocked FEV1. We assume that, over the 6 months study period, the patient has done some measurements where he was not blocked
  # To avoid taking an outlier up, which is third highest FEV1 measurement
  rmax=O2_FEV1_patient['FEV1'].nlargest(3).iloc[-1]
  # Get the theoretical healthy FEV1
  healthy_fev1=O2_FEV1_patient['FEV1 Set As'].iloc[0]
  # Add the patient id, reversed max FEV1 and healthy FEV1 to the dataframe
  new_row = pd.DataFrame({'ID': [id], 'Unblocked FEV1': [rmax], 'Healthy FEV1': [healthy_fev1]})
  df_unblocked_factor = pd.concat([df_unblocked_factor, new_row])


def add_lung_damage(fig, df, lung_damage_prct):
  xmax=df['Healthy FEV1'].max(); xmin=df['Healthy FEV1'].min()
  a=1-lung_damage_prct/100
  fig.add_shape(type="line", x0=xmin, y0=a*xmin, x1=xmax, y1=a*xmax, line=dict(color="Red", width=0.5))
  # Add line legend for no lung damage
  fig.add_annotation(x=1.02*xmax, y=a*xmax, text="{}%".format(lung_damage_prct), showarrow=False, font=dict(size=10, color="Red"))

# Plot a scatter of unblocked FEV1 against healthy FEV1
fig = px.scatter(df_unblocked_factor, x='Healthy FEV1', y='Unblocked FEV1')
add_lung_damage(fig, df_unblocked_factor, 0)
add_lung_damage(fig, df_unblocked_factor, 50)
fig.update_layout(autosize=False, width=1000, height=400)
fig.update_traces(marker=dict(size=5),
                  selector=dict(mode='markers'))
fig.show()
# Save figure
fig.write_image("{}/Unblocked FEV1 vs Healthy FEV1.pdf".format(plotsdir), width=1000, height=400)